In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

%matplotlib inline

In [ ]:
! wget --no-check-certificate -P ./ -N https://s3.ap-northeast-2.amazonaws.com/mjgim/fmnist_test.csv
! wget --no-check-certificate -P ./ -N https://s3.ap-northeast-2.amazonaws.com/mjgim/fmnist_train.csv

In [ ]:
training_data = np.loadtxt("fmnist_train.csv", delimiter=",")
training_img = training_data[:,1:]
training_lab = training_data[:,0]

test_img = np.loadtxt("fmnist_test.csv", delimiter=",")

In [ ]:
print("Training set (images) shape: {shape}".format(shape=training_img.shape))
print("Training set (labels) shape: {shape}".format(shape=training_lab.shape))

print("Test set (images) shape: {shape}".format(shape=test_img.shape))

In [ ]:
sample_size = 20

idx_s = np.random.randint(0,len(training_img), sample_size)
fig, ax = plt.subplots(1, sample_size, figsize=(sample_size*2, 2))

for i in range(sample_size):
    ax[i].set_axis_off()
    ax[i].imshow(np.reshape(training_img[idx_s][i], [28,28]),cmap="gray")

In [ ]:
# Making fileque
def group_list(l, group_size):

    for i in range(0, len(l), group_size):
        yield l[i:i+group_size]

Hidden layer 구성
- CNN layer
```p
tf.layers.conv2d(inputs=, filters=, kernel_size=, padding=, activation=)
tf.layers.max_pooling2d(inputs=, pool_size=, padding=, strides=)
```
- Dropout
```p
tf.nn.dropout(x, keep_prob)
```

In [ ]:
# Set parameter

learning_rate = 
batch_size = 
n_epoch = 

In [ ]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.int32, [None])
Y_ = tf.one_hot(Y, depth=10,  dtype=tf.float32)


H = tf.layers.dense(X , 10, activation=tf.nn.softmax)

loss = - tf.reduce_mean(tf.reduce_sum(Y_ * tf.log(H), reduction_indices=[1]))

Optimizer 변경
- GradientDescentOptimizer
- MomentumOptimizer
- RMSPropOptimizer
- AdamOptimizer

In [ ]:
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

correct_prediction = tf.equal(tf.argmax(H,1), tf.argmax(Y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

cost_list = []
acc_list = []

for i in range(n_epoch):
    cost = 0.
    x_batch = group_list(training_img, batch_size)
    y_batch = group_list(training_lab, batch_size)

    for j in range(len(training_img)//batch_size):
        batch_xs = next(x_batch)
        batch_ys = next(y_batch)
        _, c = sess.run([train, loss], feed_dict={X: batch_xs, Y: batch_ys})
        cost += c/(len(training_img)//batch_size)
    cost_list.append(cost)

    acc = sess.run(accuracy, feed_dict={X: training_img, Y: training_lab})
    acc_list.append(acc)
    
    
result = sess.run(tf.argmax(H,1), feed_dict={X: test_img})
np.savetxt("fmnist_result.csv", result, delimiter=",")

In [ ]:
plt.figure(figsize=(12,10))
plt.subplot(221)
plt.title("Cost of Trining data")
plt.xlabel("Steps")
_ = plt.plot(cost_list, "c", label="Cost")
plt.legend()

plt.subplot(222)
plt.title("Accuracy of Training data")
plt.xlabel("Steps")
_ = plt.plot(acc_list, "k--", label="Accuracy")
plt.legend()
print("Accuracy of Training data : %0.5f" %acc)